# Mojo🔥 Guided Tour

Mojo is designed to be a superset of Python, and so many of the functions that
you are already familiar with are present in Mojo.  For instance, a "hello
world" program in Mojo would look exactly like it does in Python:

In [ ]:
#| CHECK: Hello Mojo!
print("Hello Mojo!")

But on top of what Python can do, Mojo provides a set of new powerful features,
and we will go over some of them in this notebook. Fore more details and
rationale for these features please refer to the Mojo Programming Manual.

This notebook assumes familiarity with Python and focuses on what's different in
Mojo. If you'd like to learn more about Python, please refer to [Python
documentation](https://docs.python.org/) or other resources.

## Basic Systems Programming Extensions

### `'let'` and ‘`var'` declarations

Inside a `'def'` in Mojo, you may assign to a name and it implicitly creates a
function scope variable just like in Python. This provides a very dynamic and
low ceremony way to write code, but is a challenge for two reasons:
1) systems programmers often want to declare that a value is immutable, and
2) they may want to control the lifetime of a value, particularly if that value
has a destructor.

To support this, Mojo supports `‘let’` and `‘var’` declarations which introduce a
new scoped runtime value: ‘`let`’ is immutable and ‘`var`’ is mutable. These
values use lexical scoping and support name shadowing:

In [ ]:
def your_function(a, b):
    let c = a
    c = b  # error: c is immutable

    if c != b:
        var c = b
        # stuff()

`‘let’` and `‘var’` declarations support type specifiers as well as patterns,
and late initialization:

In [ ]:
def foo():
    return 3.14

def your_function():
    let x: Int = 42
    let y: F64 = 17.0

    var (a, b) = (x, y)

    var z: F32
    if x != 0:
        z = 1.0
    else:
        z = foo()
    print(z)

### `'struct'` Types

‘`struct`’ declarations in Mojo are similar in many ways to classes: they
support methods, fields, operator overloading, decorators for meta programming
(TODO), etc. On the other hand, where classes are extremely dynamic with
dynamic dispatch, dynamic method swizzling, and dynamically bound instance
properties, structs are static, bound at compile time, and are stored inlined
into their container instead of being implicitly indirect and reference
counted.

Here’s a simple definition of a struct:


In [ ]:
struct MyPair:
    var first: Int
    var second: Int

    def __init__(self&, first: Int, second: Int):
        self.first = first
        self.second = second

    def __lt__(self, rhs: MyPair) -> Bool:
        return self.first < rhs.first or
              (self.first == rhs.first and
                self.second < rhs.second)

As you can see, Mojo structs are very similar to classes, the biggest
difference is that all instance properties _must_ be explicitly declared with a
`‘var’` or `‘let’` declaration. This allows the Mojo compiler to lay out and access
the value precisely in memory without indirection or other overhead. Struct
fields are bound statically: they aren’t looked up with a dictionary
indirection. As such, you cannot ‘`del`’ a method or reassign it at runtime.
This enables the Mojo compiler to perform guaranteed static dispatch, use
guaranteed static access to fields, and inline `MyPair` into the stack frame or
enclosing type that uses it without indirection or other overheads.

You might be wondering what the “`&`” means on the `self` argument: this
indicates that the value is mutable, please see the “By-Reference” arguments
section below.

### Strong type checking

Another feature of structs is that a `struct` definition defines a
compile-time-bound name, and references to that name in a type context are
treated as a strong specification for the value being defined. For example,
consider the following code:

In [ ]:
def pairTest() -> Bool:
    let p = MyPair(1, 2)
    return p < 4 # gives a compile time error

If you attempt to run this code, you’ll get a compile time error telling you
that “`4`” cannot be converted to `MyPair`, which is what the rhs of `__lt__`
requires.


### Overloaded functions & methods

While strong type checking is good for predictability, control, and safety, it
forces you to keep the type checker happy. This can be a challenge when you
want to define expressive APIs that “just work” because some methods should
accept many different static types, and shouldn’t require the user of the API
to remember different names for all the different use cases.

To solve this problem, Mojo offers full support for “overloaded methods”. This
is a common feature seen in many programming languages (including C++, Java,
Swift, etc) where you can define the same function name with multiple different
signatures. When resolving a function call, Mojo will try each candidate and
use the one that works (if only one works), pick the closest match (if it can
determine a close match) or report the call as being ambiguous if it can’t
figure out which one to pick. In the latter case, you can resolve the ambiguity
by adding an explicit cast on the call site. Let’s look at an example:

In [ ]:
struct Array[T: AnyType]:
    fn __getitem__(self, idx: Int) -> T: ...
    fn __getitem__(self, idx: Range) -> ArraySlice: ...

The compiler follows the same approach as (e.g.) a C++ compiler does to resolve
which candidate to pick for a given call: it filters out candidates that cannot
work for a callsite, then ranks the remaining ones based on the number of
implicit conversions that are required. If there is an ambiguity, the compiler
reports an error. Mojo doesn’t support overloading solely on result type, and
doesn’t use result type or contextual type information for type inference,
keeping things simple, fast, and predictable.

### `'fn'` Definitions

The extensions above are the cornerstone that provides low-level programming
and provide abstraction capabilities, but many systems programmers prefer more
control and predictability than what ‘`def`’ in Mojo provides. To recap, ‘`def`’ is
defined by necessity to be very dynamic, flexible and generally compatible with
Python: arguments are mutable, local variables are implicitly declared on first
use, and scoping isn’t enforced. This is great for high level programming and
scripting, but is not always great for systems programming. To complement this,
Mojo provides an ‘`fn`’ declaration which is like a “strict mode” for ‘`def'`.

‘`fn`’ and ‘`def`’ are always interchangeable from an interface level: there is
nothing a ‘`def`’ can provide that a ‘`fn`’ cannot (or vice versa). The
difference is that a ‘`fn`’ is more limited and controlled on the _inside_ of
its body (alternatively, pedantic and strict). Specifically, ‘`fn`’s have a
number of limitations compared to ‘`def`’s:

1. Argument values default to being immutable in the body of the function (like
a ‘`let`’), instead of mutable (like a ‘`var`’). This catches accidental
mutations, and permits the use of non-copyable types as arguments.

2. Argument values require a type specification (except for `self` in a
method), catching accidental omission of type specifications. Similarly, a
missing return type specifier is interpreted as returning `None` instead of an
unknown return type. Note that both can be explicitly declared to return
`'object'`, which allows one to opt-in to the behavior of a `def` if desired.

3. Implicit declaration of local variables is disabled, so all locals must be
declared. This catches name typos and dovetails with the scoping provided by
`‘let’` and `‘var’`.

4. Both support raising exceptions, but this must be explicitly declared on a
`‘fn’` with the `@raises` decorator.

### The`__copyinit__` Special Method

Mojo supports full “value semantics” as seen in languages like C++, and more
advanced support than languages like Swift and Rust because it supports
non-movable types like Atomic. This is accessed by implementing special methods
like `__init__` and `__del__` on structs, which give control over the lifetime
of the logical value maintained by that struct. For example, consider a dynamic
string type that needs to allocate memory for the string data when constructed
and destroy it when the value is destroyed:

In [ ]:
struct MyString:
    var data: Pointer[Int8]

    # StringRef has a data + length field
    def __init__(self&, input: StringRef):
        let data = Pointer[Int8].alloc(input.length+1)
        data.memcpy(first.data, input.length)
        data[input.length] = 0
        self.data = Pointer[Int8](data)

    def __del__(self):
        self.data.free()


This string type is implemented in terms of traditional C `malloc()`/`free()`
calls to show a simple example of how this works. A more realistic string type
would use inline allocation and other fancy tricks to reduce allocation
overhead in standard cases, but this is simple and demonstrates the issue.

If you go ahead and try this out, you might be surprised:

In [ ]:
fn useStrings():
    var a: MyString = "hello"
    print(a)   # Should print "hello"
    var b = a  # ERROR: MyString doesn't implement __copyinit__

    a = "Goodbye"
    print(b)   # Should print "hello"
    print(a)   # Should print "Goodbye"

The compiler isn’t allowing us to make a copy of our string: MyString contains
an instance of Pointer (which is equivalent to a low-level C pointer), and Mojo
can’t know “what the pointer means” or “how to copy it” - this is one reason
why application level programmers should use higher level types like arrays and
slices! More generally, some types (like atomic numbers) cannot be copied or
moved around at all, because their address provides an **identity** just like a
class instance does.

In this case, we do want our string to be copyable around, to enable this, we
implement the `__copyinit__` special method, which is conventionally
implemented like this:

In [ ]:
struct MyString:
    var data: Pointer[Int8]

    # StringRef has a data + length field
    def __init__(self&, input: StringRef):
        let data = Pointer[Int8].alloc(input.length+1)
        data.memcpy(first.data, input.length)
        data[input.length] = 0
        self.data = Pointer[Int8](data)

    def __del__(self):
        self.data.free()

    def __copyinit__(self&, existing: Self):
        self.data = Pointer(strdup(self.data.address))

With this implementation, our code above works correctly and the “b = a” copy
produces a logically distinct instance of the string with its own lifetime and
data. The copy is made with the C strdup`()` function as instructed by the
lines of code above.

Mojo provides full control over the lifetime of a value, including the ability
to make types copyable, move-only, and not-movable. This is more control than
languages like Swift and Rust, which require values to at least be movable. If
you are curious how `existing` can be passed into the `__copyinit__` method
without itself creating a copy, check out the section on "Borrowed" argument
convention below.

## Parameterization: Compile time meta-programming

Mojo supports a full compile-time metaprogramming functionality built into the
compiler as a separate stage of compilation - after parsing, semantic analysis,
and IR generation, but before lowering to target-specific code. It uses the same
host language for runtime programs as it does for metaprograms, and leverages
MLIR to represent and evaluate these programs in a predictable way.

Let’s take a look at some simple examples.

### Defining parameterized types and functions

Mojo structs and functions may each be parameterized, but an example can help
motivate why we care. Let’s look at a
“[SIMD](https://en.wikipedia.org/wiki/Single_instruction,_multiple_data)” type,
which represents a low-level vector register in hardware that holds multiple
instances of a scalar data-type. Hardware accelerators these days are getting
exotic datatypes, and it isn’t uncommon to work with CPUs that have 512-bit or
longer SIMD vectors. There is a lot of diversity in hardware (including many
brands like SSE, AVX-512, NEON, SVE, RVV, etc) but many operations are common
and used by numerics and ML kernel developers - this type exposes them to Mojo
programmers.

Here is a (cut down) version of the SIMD API in the Mojo standard library:

In [ ]:
struct SIMD[type: DType, size: Int]:
    var value: … # Some low-level MLIR stuff here

    # Create a new SIMD from a number of scalars
    fn __init__(self&, *elems: SIMD[type, 1]):  ...

    # Fill a SIMD with a duplicated scalar value.
    @staticmethod
    fn splat(x: SIMD[type, 1]) -> SIMD[type, size]: ...

    # Cast the elements of the SIMD to a different elt type.
    fn cast[target: DType](self) -> SIMD[target, size]: ...

    # Many standard operators are supported.
    fn __add__(self, rhs: Self) -> Self: ...

Parameters in Mojo are declared in square brackets using an extended version of
the [PEP695 syntax](https://peps.python.org/pep-0695/). They are named and have
types like normal values in a Mojo program, but they are evaluated at compile
time instead of runtime by the target program. The runtime program may use the
value of parameters - because the parameters are resolved at compile time
before they are needed by the runtime program - but the compile time parameter
expressions may not use runtime values.

In the case of the `SIMD` excerpt above, there are three declared parameters:
the SIMD struct is parameterized by a ‘`size`’ parameter and a ‘`type`’
parameter. The ‘`cast`’ method is further parameterized with a ‘`target`’
parameter. Because SIMD is a parameterized type, the type of a ‘self’ argument
carries the parameters - the full type name is “`SIMD[type, size]`”. While it
is always valid to write this out (as shown in the return type of `splat`),
this can be verbose: we recommend using the `Self` type (from
[PEP673](https://peps.python.org/pep-0673/)) like the `__add__` example does.

### Using parameterized types and functions

For this type, the ‘size’ specifies the number of elements in a SIMD vector and
the type specifies the element type - for example, you might use a “4xFloat” to
represent a small floating point vector, or a “32xbfloat16’s” on an AVX-512
system with the ‘bfloat16’ machine learning type:

In [ ]:
fn funWithSIMD():
    # Make a vector of 4 floats.
    let smallVec = SIMD[DType.f32, 4](1.0, 2.0, 3.0, 4.0)

    # Make a big vector containing 1.0 in bfloat16 format.
    let bigVec = SIMD[DType.bf16, 32].splat(1.0)

    # Do some math and convert the elements to float32.
    let biggerVec = (bigVec+bigVec).cast[DType.f32]()

    # You can write types out explicitly if you want of course.
    let biggerVec2 : SIMD[DType.f32, 32] = biggerVec


Note that the “cast” method needs an additional parameter to indicate what type
to cast to: that is handled by parameterizing the call to “cast”. The example
above shows the use of concrete types, but the major power of parameters comes
from the ability to define parametric algorithms and types, e.g. it is quite
easy to define parametric algorithms, e.g. ones that are length- and
DType-agnostic:


In [ ]:
fn rsqrt[width: Int, dt: DType](x: SIMD[dt, width]) -> SIMD[dt, width]:
    return 1 / sqrt(x)

The Mojo compiler is fairly smart about type inference with parameters. Note
that this function is able to call the parametric “`sqrt(x)`” function without
specifying the parameters, the compiler infers its parameters as if you wrote
“`sqrt[width,type](x)`” explicitly. Also note that “`rsqrt`” chose to define
its first parameter named “width” but the SIMD type names it “`size`” without
challenge.

### Parameters expressions are just Mojo code

All parameters and parameter expressions are typed using the same type system
as the runtime program: ‘Int’ and ‘DType’ are implemented in the Mojo standard
library as structs. Parameters are quite powerful, supporting the use of
expressions with operators, function calls etc at compile time, just like a
runtime program. This enables the use of many ‘dependent type’ features, for
example, you might want to define a helper function to concatenate two SIMD
vectors:

In [ ]:
fn concat[len1: Int, len2: Int, ty: DType](
    lhs: SIMD[ty, len1], rhs: SIMD[ty, len2]) -> SIMD[len1+len2, ty]:
        ...

fn use_vectors(a: SIMD[DType.f32, 4], b: SIMD[DType.f16, 8]):
    let x = concat(a, a)  # Length = 8
    let y = concat(b, b)  # Length = 16


Note how the result length is the sum of the input vector lengths, and you can
express that with a simple + operation. In practice, the SIMD type only allows
power of two length vectors, so this isn’t actually needed in this example, but
having a powerful and expressive type equations can enable powerful and elegant
API designs. For a more complex example, take a look at the `SIMD.shuffle`
method in the standard library: it takes two input SIMD values, a vector
shuffle mask as a list, and returns a SIMD that matches the length of the
shuffle mask.

### Powerful Compile-time Programming

While simple expressions are useful, sometimes you want to write imperative
compile-time logic with control flow. For example, the “isclose” function in
Math.mojo uses exact equality for integers but “close” comparison for floating
point. You can even do compile time recursion, e.g. here is an example “tree
reduction” algorithm that sums all elements of a vector recursively into a
scalar:

In [ ]:
struct SIMD[size: Int, type: DType]:
    ...
    fn reduce_add(self) -> SIMD[type, 1]:
        @parameter
        if size == 1:
            return self[0]
        elif size == 2:
            return self[0] + self[1]

        # Extract the top/bottom halves, add them, sum the elements.
        alias half_size = size // 2
        let lhs = self.slice[half_size](0)
        let rhs = self.slice[half_size](half_size)
        return (lhs + rhs).reduce_add()

This makes use of two new features: `'@parameter if'` is an if statement that
runs at compile time. It requires that its condition be a valid parameter
expression, and ensures that only the live branch of the if is compiled into
the program. This example also introduces the `'alias'` declaration, which is
like `‘let’` but which is guaranteed to be evaluated at compile time.


### Mojo Types are just Parameter Expressions

While we’ve shown how you can use parameter expressions within types, in both
Python and Mojo, type annotations can themselves be arbitrary expressions.
Types in Mojo have a special metatype type, allowing type-parametric algorithms
and functions to be defined, for example one can define an algorithm like the
C++ `std::vector` class like this:

In [ ]:
struct DynamicVector[type: AnyType]:
        ...
    fn reserve(self&, new_capacity: Int): ...
    fn push_back(self&, value: type): ...
    fn pop_back(self&): ...
    fn __getitem__(self, i: Int) -> type: ...
    fn __setitem__(self&, i: Int, value: type): ...

fn use_vector():
    var v = DynamicVector[Int]()
    v.push_back(17)
    v.push_back(42)
    v[0] = 123
    print(v[1])      # Prints 42
    print(v[0])      # Prints 123

Notice that the ‘type’ parameter is being used as the formal type for the
‘value’ arguments and the return type of the `__getitem__` function. Parameters
allow the `DynamicVector` type to provide different APIs based on the different
use-cases. There are many other cases that benefit from more advanced use
cases. For example, the parallel processing library defines the
parallelForEachN algorithm, which executes a closure N times in parallel,
feeding in a value from the context. That value can be of any type:

In [ ]:
fn parallelize[
    func: Function[(Int), None]
](rt: Runtime, num_work_items: Int):
    # Not actually parallel: see Functional.mojo for real impl.
    for i in range(num_work_items):
        func(i)

This is possible because the ‘func’ parameter is allowed to refer to the
earlier ‘arg_type’ parameter, and that refines its type in turn.

We don’t have full support for variadic parameter lists, but we expect this to
work soon:

In [ ]:
struct Tuple[*ElementTys: AnyType]:
    var _storage : ElementTys

which will allow us to fully define Tuple (and related types like Function) in
the standard library.

### `alias`: Named Parameter Expressions

It is very common to want to name compile time values. Whereas ‘var’ defines a
runtime value, and ‘let’ defines a runtime constant, we need a way to define a
compile time temporary value. For this, Mojo uses an ‘`alias`’ declaration. For
example, the DType struct implements a simple enum using aliases for the
enumerators like this (the actual internal implementation details vary a bit):

In [ ]:
struct DType:
    var value : Int8
    alias invalid = DType(0)
    alias bool = DType(1)
    alias si8 = DType(2)
    alias ui8 = DType(3)
    alias si16 = DType(4)
    alias ui16 = DType(5)
    ...
    alias f32 = DType(15)


This allows clients to use ‘DType.f32’ as a parameter expression (which also
works as a runtime value of course) naturally. Note that this is invoking the
runtime constructor for DType at compile time.

Type are another common use for alias: because types are just compile time
expressions, it is very handy to be able to do things like this:

In [ ]:
alias F32 = SIMD[DType.f32, 1]
alias UI8 = SIMD[DType.ui8, 1]

var x : F32   # F32 works like a "typedef"

Like `‘var’` and `‘let’`, aliases obey scope and you can use local aliases
within functions as you’d expect.


### Autotuning / Adaptive compilation

TBD

## Argument Passing Control and Memory Ownership


In both Python and Mojo, much of the language revolves around function calls: a
lot of the (apparently) built-in functionality is implemented in the standard
library with “dunder” methods. Mojo takes this a step further than Python, by
putting the most basic things (like integers and the object type itself) into
the standard library.

### Why argument conventions are important

In Python all fundamental values are references to objects - a Python
programmer typically thinks about the programming model as everything being
reference semantic. However, at the CPython or machine level, we can see that
the references themselves are actually passed _by-copy_, by copying the pointer
and adjusting reference counts.

Mojo on the other hand provides full control over value copies, aliasing of
references, and mutations.

### By-Reference Arguments

Let’s start with the simple case: passing mutable references to values vs
passing immutable references. As we already know, arguments that are passed to
fn’s are immutable by default:

In [ ]:
struct Int:
    # self and rhs are both immutable in __add__.
    fn __add__(self, rhs: Int) -> Int: ...

    # ... but this cannot work for __iadd__
    fn __iadd__(self, rhs: Int):
        self = self + rhs  # ERROR: cannot assign to self!


The problem here is that `__iadd__` needs to mutate the internal state of the
integer. The solution in Mojo is to declare that the argument is passed “by
reference” by using the & marker on the argument name (self in this case):

In [ ]:
struct Int:
    # ...
    fn __iadd__(self&, rhs: Int):
        self = self + rhs    # OK


Because this argument is passed by-reference, the ‘self’ argument is mutable in
the callee, and any changes are visible in the caller - even if the caller has
a non-trivial computation to access it, like an array subscript:


In [ ]:

fn show_mutation():
    var x = 42
    x += 1
    print(x)    # prints 43 of course

    var a = InlinedFixedVector[16, Int](...)
    a[4] = 7
    a[4] += 1
    print(a[4])  # Prints 8

    let y = x
    y += 1       # ERROR: Cannot mutate 'let' value

Mojo implements the in-place mutation of the InlinedFixedVector element by
emitting a call to `__getitem__` into a temporary buffer, followed by a store
with `__setitem__` after the call. Mutation of the ‘let’ value fails because it
isn’t possible to form a mutable reference to an immutable value. Similarly,
the compiler rejects attempts to use a subscript with a by-ref argument if it
implements `__getitem__` but not `__setitem__`.

There is nothing special about ‘self’ in Mojo, and you can have multiple
different by-ref arguments. For example, you can define and use a swap function
like this:


In [4]:
fn swap(lhs&: Int, rhs&: Int):
  let tmp = lhs
  lhs = rhs
  rhs = tmp

fn show_swap():
    var x = 42
    var y = 12
    swap(x, y)
    print(x)  # Prints 12
    print(y)  # Prints 42

### “Borrowed” Argument Convention


Now that we know how by-reference argument passing works, you may wonder how
by-value argument passing works and how that interacts with the `__copyinit__`
method which implements copy constructors. In Mojo, the default convention for
passing arguments to functions is to pass with the “borrowed” argument
convention. You can spell this out explicitly if you’d like:


In [ ]:

fn useSomethingBig(borrowed a: SomethingBig, b: SomethingBig):
    """'a' and 'b' are passed the same, because 'borrowed' is the default."""
    a.print_id()
    b.print_id()

This default applies to all arguments uniformly, including the `self` argument
of methods. The borrowed convention passes an _immutable reference_ to the
value from the caller’s context, instead of copying the value. This is much
more efficient when passing large values, or when passing expensive values like
a reference counted pointer (which is the default for Python/Mojo classes),
because the copy constructor and destructor don’t have to be invoked when
passing the argument. Here is a more elaborate example building on the code
above:


In [ ]:
# A type that is so expensive to copy around we don't even have a
# __copyinit__ method.
struct SomethingBig:
    var id_number: Int
    var huge: InlinedArray[Int, 100000]
    fn __init__(self&): …

    # self is passed by-reference for mutation as described above.
    fn set_id(self&, number: Int):
        self.id_number = number

    # Arguments like self are passed as borrowed by default.
    fn print_id(self):  # Same as: fn print_id(borrowed self):
        print(self.id_number)

fn try_something_big():
    # Big thing sits on the stack: after we construct it it cannot be
    # moved or copied.
    let big = SomethingBig()
    # We still want to do useful things with it though!
    big.print_id()
    # Do other things with it.
    useSomethingBig(big, big)


Because the default argument convention is borrowed, we get very simple and
logical code which does the right thing by default: for example, we don’t want
to copy all of SomethingBig just to invoke the “`print_id`” method, or when
calling `useSomethingBig`.

The borrowed convention is similar and has precedent to other languages. For
example, the borrowed argument convention is similar in some ways to passing an
argument by “`const&`” in C++. This avoids a copy of the value, and disables
mutability in the callee. The borrowed convention differs from “`const&`” in
C++ in two important ways though:

1. The Mojo compiler implements a borrow checker (similar to Rust) that
prevents code from dynamically forming mutable references to a value when there
are immutable references outstanding, and prevents having multiple mutable
references to the same value. You are allowed to have multiple borrows (as the
call to “`useSomethingBig`” does above) but cannot pass something by mutable
reference and borrow at the same time.

2. Small values like “`Int`”, “`Float`”, and “`SIMD`” are passed directly in
machine registers instead of through an extra indirection (this is because they
are declared with the “`@register_passable`” decorator, see below). This is a
[significant performance
enhancement](https://www.forrestthewoods.com/blog/should-small-rust-structs-be-passed-by-copy-or-by-borrow/)
when compared to languages like C++ and Rust, and moves this optimization from
every call site to being declarative on a type.

Rust is another important language and the Mojo and Rust borrow checkers
enforce the same exclusivity invariants. The major difference between Rust and
Mojo is that no sigil is required on the caller side to pass by borrow, Mojo is
more efficient when passing small values, and Rust defaults to moving values by
default instead of passing them around by borrow. These policy and syntax
decisions allows Mojo to provide an arguably easier to use programming model.


### “Owned” Argument Convention
TBD


### `@register_passable` Struct Decorator

As described above, the default fundamental model for working with values is
that they live in memory so they have identity, which means they are passed
indirectly to and from functions (equivalently, they are passed ‘by reference’
at the machine level). This is great for types that cannot be moved, and is a
good safe default for large objects or things with expensive copy operations.
However, it is really inefficient for tiny things like a single integer or
floating point number!

To solve this, Mojo allows structs to opt-in to being passed in a register
instead of passing through memory with the `@register_passable` decorator.
You’ll see this decorator on types like Int in the standard library:


In [ ]:
@register_passable
struct Int:
   var value: __mlir_type.`!pop.scalar<index>`

   fn __copyinit__(self&, existing: Self):
       return Self {value: self.value}

   fn __init__(value: __mlir_type.`!pop.scalar<index>`) -> Self:
       return Self {value: value}
   ...


This decorator does not change the fundamental behavior of a type: it still
needs to have a `__copyinit__` method to be copyable, may still have a
`__init__` and `__del__` methods, etc. The major effect of this decorator is on
internal implementation details: `@register_passable` types are typically
passed in machine registers (subject to the details of the underlying
architecture of course).

There are only a few observable effects of this decorator to the typical Mojo
programmer:

1. `@register_passable` types are not being able to hold instances of types
that are not themselves `@register_passable`.

2. instances of `@register_passable` types do not have predictable identity,
and so the ‘self’ pointer is not stable/predictable (e.g. in hash tables).

3. `@register_passable`arguments and result are exposed to C and C++ directly,
instead of being passed by-pointer.

4. The `__init__` and `__copyinit__` methods of this type are implicitly static
(like `__new__` in Python) and returns its result by-value instead of taking
`self&`.

We expect that this decorator will be used pervasively on core standard library
types, but is safe to ignore for general application level code.


## Advanced Mojo Features

This section describes power-user features that are important for building the
bottom-est level of the standard library. This level of the stack is inhabited
by narrow features that require experience with compiler internals to
understand and utilize effectively.

### `@always_inline` decorator


TBD

### Direct Access to MLIR


TBD